In [3]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Goal: Estimate acceleration profile $\hat{m}$ Using Newton-Raphson to compensate for distortion between similar point cloud

Through the first order Taylor approximation we define: 

$\mathbf{h}(\hat{m})$ as the means of voxel centers in cloud $j$ passed through the distortion operator $\mathbb{C}$ according to estimated motion profile $\hat{m}$. 

$\mathbf{H}$ is the Jacobian of $\mathbf{h}$, which is multiplied by a linear correction factor $\delta m$.

$    y_i = \mathbf{h}(\hat{m}) + \mathbf{H} \delta m $


In [747]:
#load point cloud
# no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# m_hat = np.array([-3., -2., 0., 0., 0., 0.0])

# movement in x
old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
m_hat = np.array([3, 0, 0., 0., 0., 0])
gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# # movement in x, y, & yaw
# old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
# m_hat = np.array([3, -1, 0., 0., 0., -1])
# gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# # print(gt) 

#downsample
old_cloud = old_cloud[::5,:]

In [748]:
from remove_motion_basic import linear_correction_old as lc 

new_cloud = lc(old_cloud, m_hat)
# undistorted = lc(new_cloud, -m_hat)

In [749]:
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
# disp.append(Points(undistorted, c = "#CB2314"))
# disp.append(Points(old_cloud, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Analytically calculating derivatives with SymPy

In [937]:
from sympy import *

x, y, z = symbols('x y z')

init_printing(use_unicode=True)

In [264]:
#sovles for the derivative
ans1 = diff(cos(x), x)
print(ans1)

#creates an unevaluated derivative
ans2 = Derivative(cos(x), x)
print(ans2)

-sin(x)
Derivative(cos(x), x)


In [265]:
ans3 = integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)
# print(latex(ans3)) #-> converts to LaTeX so I don't have to re-copy everything 

integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)


In [274]:
a = Matrix([[10*x, 2], 
            [3, 4*x]])
# A = integrate(a**2)
# print(latex(A))
integrate(a**2)

⎡     3                   ⎤
⎢100⋅x                2   ⎥
⎢────── + 6⋅x     14⋅x    ⎥
⎢  3                      ⎥
⎢                         ⎥
⎢                  3      ⎥
⎢       2      16⋅x       ⎥
⎢   21⋅x       ───── + 6⋅x⎥
⎣                3        ⎦

In [288]:
b = Matrix([[x + 1, 2*y],
            [3*x**2*y, 3 + x + y**2]])
diff(b, y, x)

⎡ 0   0⎤
⎢      ⎥
⎣6⋅x  0⎦

# Derive Jacobain H

### $\mathbf{H} \in \mathbb{R}^{4N \times 6} $

### $\mathbf{H} = 
\begin{bmatrix}
^{(n)}H_{x} ~ ^{(n)}H_{y} ~ ^{(n)}H_{z} ~ ^{(n)}H_{\phi} ~ ^{(n)}H_{\theta} ~ ^{(n)}H_{\psi}
\end{bmatrix}$

### $ ^{(n)}\mathbf{H}_x =  \frac{\delta{\mathbb{C}}}{\delta x} ~ ^{(n)}y_j \in \mathbb{R}^{4 \times 1}$

### Here $^{(n)}y_j$ refers to distribution mean from scan j inside voxel n, denoted in homogenous coordinates

$[x, y, z] \rightarrow [x, y, z, 1] $ 

<!-- <span style="color:red"> TODO: figure out if I need to use homogeneous coordinates for $y_j$ </span> -->

# Calculate correction  function $\mathbb{C}$ 

## $\mathbb{C}(\hat{m}) = T_{\text{rect}}(M(\hat{m})) \in \mathbb{R}^{4 \times 4} $

Here, the rectifying transformation $T_{\text{rect}}$ for each point in $y_j$ depends on scaled values of $\hat{m}$ which are concatenated to form matrix $M$. Since points are stored in the order they are recieved and are somewhat even spread throughout the scene, we can approximate $M$ through a simple linspace matrix without needing to transform to a spherical representation. $t_{\text{scale}}$ is a scaling parameter to normalize the turn rate of the sensor in the presence of composite yaw rotation.

## $
M = \bigg{(} \{ r: r = t_{\text{scale}} \frac{p}{P}, p \in \{ 0, 1, ..., P \} \} \bigg{)} \hat{m}
\in \mathbb{R}^{P \times 6}
% \begin{bmatrix}
% 1 & 1 & 1 & 1 
% \end{bmatrix}
$

## $t_{\text{scale}} = \frac{2\pi}{-\psi + \omega_{\text{LIDAR}} } $ 


My old way of "unwinding" was acomplished by converting the point cloud to spherical coordinates, rescaling the yaw angle, and then converting back to Cartesian 

<!-- ## $y_{j, \text{unwound}} =
\text{s2c} \bigg{(} \text{c2s}(y_{j})
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & \psi_{\text{rect}}
\end{bmatrix} \bigg{)}$
 -->
Unfortunately, c2s() is a nonlinear operation so it makes things a little tricky when we need to do everything via linear algebra

<!-- ## $ \text{c2s}(y_j) = 
\begin{bmatrix}
x_1 & y_1 & z_1 \\
x_2 & y_2 & z_2 \\
 & \vdots &
\end{bmatrix}
\rightarrow
\begin{bmatrix}
r_1 & \phi_1 & \theta_1 \\
r_2 & \phi_2 & \theta_2 \\
r_3 & \phi_3 & \theta_3
\end{bmatrix}
$
 -->
<!-- ### GPT:
lol it really got this wro

$ ^{(n)}y_j = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}
$

$ B = \begin{bmatrix}
\text{sin}(\theta) ~ \text{cos}(\phi) & \text{cos}(\theta) ~ \text{cos}(\phi) & -\text{sin}(\phi) \\
\text{sin}(\theta)\text{sin}(\phi) & \text{cos}(\theta)\text{sin}(\phi) & \text{cos}(\phi) \\
\text{cos}(\theta) & -\text{sin}(\theta) & 0
\end{bmatrix}
$

$B ~ ^{(n)}y_j = ^{(n)}y_{j, \text{spherical}} $ -->


<!-- Putting everything together:

 $
\mathbb{C}(y_j, \hat{m}) =
M ~ T_{\text{rect}} \bigg{[} \text{s2c} \bigg{(} \text{c2s}(y_{j})
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & \psi_{\text{rect}}
\end{bmatrix} \bigg{)},
\begin{bmatrix}
    1 \\
    \vdots 
\end{bmatrix}
\bigg{]}^T$
 -->

In [656]:
period_lidar = 1
m_hat = np.array([3, 0, 0., 0., 0., -0.5])
t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
lsvec = np.linspace(0,t_scale, 100)

print(np.shape(m_hat))
M = m_hat * np.array([lsvec, lsvec, lsvec, lsvec, lsvec, lsvec]).T
# print(np.shape(M))
# print(M)

(6,)


## Get transform $T_{\text{rect}}$

In [938]:
x, y, z, phi, theta, psi = symbols('x y z phi theta psi')
# m_hat = Matrix([x, y, z, phi, theta, phi]).T
m_hat = np.array([1,2,3,0.4,0.5,0.6])
print( "m_hat = ", pretty(m_hat))

T_roll = Matrix([[1, 0, 0, 0],
                 [0, cos(phi), sin(phi), 0],
                 [0, -sin(phi), cos(phi), 0],
                 [0, 0, 0, 1]])
# print("\n T_roll =") 
# print(pretty(T_roll))

T_pitch = Matrix([[cos(theta), 0, -sin(theta), 0],
                 [0, 1, 0, 0],
                 [sin(theta), 0, cos(theta), 0],
                 [0, 0, 0, 1]])
# print("\n T_pitch =") 
# print(pretty(T_pitch))

#Recall we are setting T_yaw to identity here since scan axis-aligned motion is already compensated for
T_yaw = Matrix([[cos(psi), sin(psi), 0, 0],
                 [-sin(psi), cos(psi), 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
# print("\n T_yaw =") 
# print(pretty(T_yaw))

T_trans = Matrix([[1, 0, 0, x],
                 [0, 1, 0, y],
                 [0, 0, 1, z],
                 [0, 0, 0, 1]])
# print("\n T_trans =") 
# print(pretty(T_trans))

# T_rect = (T_roll * T_pitch * T_yaw * T_trans)
T_rect = (T_roll * T_pitch * T_yaw * T_trans).inv()
print("T_rect =")
T_rect


m_hat =  [1.  2.  3.  0.4 0.5 0.6]
T_rect =


⎡                                                                             
⎢                            cos(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                                                                             
⎢                            sin(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                               -sin(θ)             

In [964]:
# print(diff(T_rect, x))
# diff(T_rect,y)
# print(diff(T_rect, z))
print(diff(T_rect, psi))
# diff(T_rect, phi)
diff(T_rect, psi)


Matrix([[-sin(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (-sin(phi)*sin(psi)*sin(theta) - sin(theta)**2*cos(phi)*cos(psi) - cos(phi)*cos(psi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (sin(phi)*sin(theta)**2*cos(psi) + sin(phi)*cos(psi)*cos(theta)**2 - sin(psi)*sin(theta)*cos(phi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), 0], [cos(psi)*co

⎡                                                                             
⎢                           -sin(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                                                                             
⎢                            cos(ψ)⋅cos(θ)                                    
⎢─────────────────────────────────────────────────────────────────────  ──────
⎢   2       2         2       2         2       2         2       2        2  
⎢sin (ψ)⋅sin (θ) + sin (ψ)⋅cos (θ) + sin (θ)⋅cos (ψ) + cos (ψ)⋅cos (θ)  sin (φ
⎢                                                                             
⎢                                  0                

In [903]:
# print(T_rect) # -> take printed output and convert to numpy array

# from tensorflow.math import sin, cos
from numpy import sin, cos


# x = 1
# y = 2
# z = 3
# phi = 0.1
# theta = 0.2
# psi = 0.3

# x = tf.transpose(tf.constant([[1.,2.,3.,4.,5.]], tf.float64))
# y = tf.transpose(tf.constant([[1.,2.,3.,4.,5.]], tf.float64))
# z = tf.transpose(tf.constant([[1.,2.,3.,4.,5.]], tf.float64))
# phi = tf.transpose(tf.constant([[1.,2.,3.,4.,5.]], tf.float64))
# theta = tf.transpose(tf.constant([[1.,2.,3.,4.,5.]], tf.float64))
# psi = tf.transpose(tf.constant([[1.,2.,3.,4.,5.]], tf.float64))

# print(x)

x = np.array([1,2,3])#[:,None]
y = np.array([1,2,3])#[:,None]
z = np.array([1,2,3])#[:,None]
phi = np.array([1,2,3])#[:,None]
theta = np.array([1,2,3])#[:,None]
psi = np.array([1,2,3])#[:,None]
test = np.array([cos(x) * cos(y) / sin(phi), cos(x) * cos(y) / sin(phi)])
print(np.shape(test))
print(test)

# T_rect_np = tf.Variable([[cos(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(theta)*cos(psi) - sin(psi)*sin(theta)**2*cos(phi) - sin(psi)*cos(phi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta)**2 + sin(phi)*sin(psi)*cos(theta)**2 + sin(theta)*cos(phi)*cos(psi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -x], [sin(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta) + sin(theta)**2*cos(phi)*cos(psi) + cos(phi)*cos(psi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (-sin(phi)*sin(theta)**2*cos(psi) - sin(phi)*cos(psi)*cos(theta)**2 + sin(psi)*sin(theta)*cos(phi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -y], [-sin(theta)/(sin(theta)**2 + cos(theta)**2), sin(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), cos(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), -z], [0, 0, 0, 1]])

T_rect_np = np.array([[cos(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(theta)*cos(psi) - sin(psi)*sin(theta)**2*cos(phi) - sin(psi)*cos(phi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta)**2 + sin(phi)*sin(psi)*cos(theta)**2 + sin(theta)*cos(phi)*cos(psi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -x], [sin(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta) + sin(theta)**2*cos(phi)*cos(psi) + cos(phi)*cos(psi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (-sin(phi)*sin(theta)**2*cos(psi) - sin(phi)*cos(psi)*cos(theta)**2 + sin(psi)*sin(theta)*cos(phi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -y], [-sin(theta)/(sin(theta)**2 + cos(theta)**2), sin(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), cos(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), -z], 
                      [np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.ones(len(x))]])
print(np.shape(T_rect_np))
test = np.transpose(T_rect_np, (2,0,1))
print(test)
# test = np.reshape(test, [-1, 4])
# print(test)
# print(T_rect_np)
# print(T_rect_np[0])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
(2, 3)
[[0.34692412 0.19045274 6.94504739]
 [0.34692412 0.19045274 6.94504739]]
(4, 4, 3)
[[[ 0.29192658 -0.07207501  0.95372117 -1.        ]
  [ 0.45464871  0.88774982 -0.07207501 -1.        ]
  [-0.84147098  0.45464871  0.29192658 -1.        ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.17317819  0.03432197  0.98429229 -2.        ]
  [-0.37840125  0.92500513  0.03432197 -2.        ]
  [-0.90929743 -0.37840125  0.17317819 -2.        ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.98008514  0.11999219  0.15822448 -3.        ]
  [-0.13970775  0.98289553  0.11999219 -3.        ]
  [-0.14112001 -0.13970775  0.98008514 -3.        ]
  [ 0.          0.          0.          1.        ]]]


In [935]:
y_j = y_j[:3]
if np.shape(y_j)[1] == 3:
    y_j = np.append(y_j, np.ones([len(y_j),1]), axis = 1)

# y_j_tf = tf.convert_to_tensor(y_j[:,:,None])
# test_tf = tf.convert_to_tensor(test)
# print(np.shape(y_j_tf))
# print(np.shape(test_tf))

# ans = y_j_tf * test_tf

print(np.shape(y_j[:,:,None]))
print(np.shape(test))
# ans = y_j[:,:,None] * test
ans = test @ y_j[:,:,None]


print(np.shape(ans))
print(ans)
ans = np.reshape(ans, (-1, 1))
print(ans)

(3, 4, 1)
(3, 4, 4)
(3, 4, 1)
[[[ -1.69575953]
  [  0.25482605]
  [ -3.58853231]
  [  1.        ]]

 [[  0.6746111 ]
  [ -6.03234702]
  [-12.41095148]
  [  1.        ]]

 [[  8.1516287 ]
  [ -4.04420869]
  [ -5.57667941]
  [  1.        ]]]
[[ -1.69575953]
 [  0.25482605]
 [ -3.58853231]
 [  1.        ]
 [  0.6746111 ]
 [ -6.03234702]
 [-12.41095148]
 [  1.        ]
 [  8.1516287 ]
 [ -4.04420869]
 [ -5.57667941]
 [  1.        ]]


In [957]:
# diff(T_rect, x)
diff(T_rect, phi)

# dT_rect_dx = diff(T_rect, x)
# pprint(dT_rect_dx)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


⎡                                                                             
⎢                                                                        sin(φ
⎢0  ──────────────────────────────────────────────────────────────────────────
⎢      2       2       2         2       2       2         2       2       2  
⎢   sin (φ)⋅sin (ψ)⋅sin (θ) + sin (φ)⋅sin (ψ)⋅cos (θ) + sin (φ)⋅sin (θ)⋅cos (ψ
⎢                                                                             
⎢                                                                             
⎢                                                                       - sin(
⎢0  ──────────────────────────────────────────────────────────────────────────
⎢      2       2       2         2       2       2         2       2       2  
⎢   sin (φ)⋅sin (ψ)⋅sin (θ) + sin (φ)⋅sin (ψ)⋅cos (θ) + sin (φ)⋅sin (θ)⋅cos (ψ
⎢                                                                             
⎢                                                   

In [663]:
#substitue in values
expr = diff(T_rect, phi)
# print(expr)
# expr = T_rect
# print(m_hat)
ans = expr.subs([(x, m_hat[0]), 
                 (y, m_hat[1]),
                 (z, m_hat[2]),
                 (phi, m_hat[3]),
                 (theta, m_hat[4]),
                 (psi, m_hat[5])])
print(type(np.array(ans)))
print("\n", np.array(ans).astype(np.float64))

<class 'numpy.ndarray'>

 [[ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]]


# Test rectifying motion distortion usuing SymPy Linalg technique

In [955]:
from remove_motion_basic import get_H
from remove_motion_basic import linear_correction_old as lc 

m_hat = np.array([3, 0, 0, 0, 0, 0])
skip = 1
y_j = old_cloud[::skip]
y_i = new_cloud[::skip] #has distortion already corrected
H = get_H(y_j = y_j, m_hat = m_hat )

#plot result
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_j, c = "#CB2314", r=10))
disp.append(Points(y_i, c = "#3F5151", r=10))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [956]:
print(np.shape(H))
print(np.shape(y_j)[0]*4)

(72296, 6)
72296


# Simple Test using pre-rectified point cloud
#### (in this example we don't need to worry about iteratively finding correspondences)

In [753]:
#assume initial correction factor of 0 for each solution component
m_hat = np.array([0., 0, 0, 0, 0, 0])
skip = 100 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud
# y_j = old_cloud[::skip] + np.array([0, 1., 0.]) #test offset - hurts performance

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#3F5151", alpha = (count+1)/(runlen+1), r=10))

runlen = 25
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("m_hat = ", m_hat)
    
    #apply last estimate of correction
    y_j_undistort = lc(y_j, m_hat)
    print(np.shape(y_j_undistort))
    
    #delta_m =  ((H^T*H)^-1)(H^T)(yi - yj*)
    H = get_H(y_j_undistort, m_hat)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
    
    print(np.shape(residual))
    delta_m = np.linalg.pinv(H.T @ H) @ H.T @ residual
    m_hat[:3] -= delta_m[:3]
    m_hat[3:] += delta_m[3:] #TODO: figure out why angles are backwards...    
    
    #plot updated cloud2
#     color = [0.5 + count/(runlen*2), 1 - count/runlen, count/runlen]
    disp.append(Points(y_j_undistort[:,:3], c = "#CB2314", alpha = (count+1)/(runlen+1), r=10))

    
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
m_hat =  [0. 0. 0. 0. 0. 0.]
(181, 3)
(724,)
~~~~ iteration  1 ~~~~~~~~~~~
m_hat =  [ 1.49453200e+00  2.13243978e-02  1.20762254e-04 -5.77911268e-04
 -2.26279706e-04  4.27833604e-02]
(181, 3)
(724,)
~~~~ iteration  2 ~~~~~~~~~~~
m_hat =  [ 2.18209107e+00  2.75540523e-03 -8.67128012e-04 -7.29990380e-04
 -6.54331168e-04  4.20907672e-02]
(181, 3)
(724,)
~~~~ iteration  3 ~~~~~~~~~~~
m_hat =  [ 2.52489275e+00 -9.15997447e-03 -1.74965108e-03 -6.23135577e-04
 -7.88707958e-04  3.18724910e-02]
(181, 3)
(724,)
~~~~ iteration  4 ~~~~~~~~~~~
m_hat =  [ 2.71106725e+00 -1.31152264e-02 -2.05721004e-03 -4.64593038e-04
 -7.29326061e-04  2.21123034e-02]
(181, 3)
(724,)
~~~~ iteration  5 ~~~~~~~~~~~
m_hat =  [ 2.81902895e+00 -1.27322326e-02 -1.94032594e-03 -3.28564549e-04
 -5.92573150e-04  1.48042529e-02]
(181, 3)
(724,)
~~~~ iteration  6 ~~~~~~~~~~~
m_hat =  [ 2.88435160e+00 -1.06441483e-02 -1.62870457e-03 -2.28363873e-04
 -4.46230275e-04  9.77044571e-03]
(181, 3)
(724,)
~

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [719]:
print(m_hat)

[ 3.38350195e+00  9.08477967e-02  5.62851297e-03  2.89060111e-03
 -1.12482944e-02 -3.13726236e-01]


In [704]:
a = np.array([[1,2],[3,4]])
print(a)
print(a.flatten())

[[1 2]
 [3 4]]
[1 2 3 4]
